In [2]:
import os
import openai
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from openai import OpenAI

In [3]:
# https://platform.openai.com/docs/guides/optimizing-llm-accuracy/llm-optimization-context
# prompt engineering
# few-shot, zero-shot learning
# RAG - using this technique to find similar responses to use for few-shot learning

In [4]:
df = pd.read_csv('../../data/csv/toxic_classification_evaluation_set.csv')
df['Toxic'] = df['Toxic'].apply(lambda x: str(np.int64(x)))
df

,Text,Toxic
0,going handle notable alumni like Duffield atte...,0
1,wwwcornsilkscom sockpuppets,0
2,1049 11 Sep 2004 UTC,0
3,Deep web article Thanks message made change th...,0
4,Youre quite coward kutta joking However one th...,1
...,...,...
995,Go kill ashamed Twoofers like scumbags deserve...,1
996,Fuck cluelessbot filthy roodypoo99241217133,1
997,Besides defecate image ex Grave Vince Talabér ...,0
998,Gay Guy know Wuzzup like sadomasochism Gim,1


In [5]:
load_dotenv()
FINE_TUNED_TOXIC_DETECTION_API_KEY = os.getenv("FINE_TUNED_TOXIC_DETECTION_API_KEY")
REGULAR_TOXIC_DETECTION_API_KEY = os.getenv("REGULAR_TOXIC_DETECTION_API_KEY")

In [6]:
general = OpenAI(
    api_key=REGULAR_TOXIC_DETECTION_API_KEY
)

In [7]:
original = pd.DataFrame(columns=['Text', 'Toxic'])

In [8]:
evaluation_prompt = ''
# Read the evaluation prompt from the text file with utf-8 encoding
with open("../../data/text/prompt_engineering/gpt-3-5-0125-two-shot-prompt.txt", "r", encoding="utf-8") as file:
    evaluation_prompt += file.read()

In [9]:
evaluation_prompt

'Classify these comments as either toxic (1) non-toxic (0). Toxic comments contain offensive, harmful, or abusive language. Label 1 for toxic and 0 for non-toxic\n\nExamples for Reference:\n\nText: “I disagree with you, but I think you make some valid points.”\n\nLabel: 0\n\nText: “Go back to where you came from, we don’t want you here.”\n\nLabel: 1\n\nClassify the following comment:'

In [10]:
# gpt-3.5-turbo-0125
for index, row in df.iterrows():
    completion = general.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {
                "role": "system", 
                "content": evaluation_prompt
            },
            {
                "role": "user",
                "content": row['Text']
            }
        ]
    )
    original.loc[index] = [row['Text'], completion.choices[0].message.content]


In [11]:
# gpt-3.5 needs prompt to be really specific; more instruction needed
# gpt-4 will give you a response of a number as requested, gpt-3.5 will give
# a longer sequence of characters which wasn't asked for
original

,Text,Toxic
0,going handle notable alumni like Duffield atte...,0
1,wwwcornsilkscom sockpuppets,1
2,1049 11 Sep 2004 UTC,0
3,Deep web article Thanks message made change th...,0
4,Youre quite coward kutta joking However one th...,1
...,...,...
995,Go kill ashamed Twoofers like scumbags deserve...,1
996,Fuck cluelessbot filthy roodypoo99241217133,1
997,Besides defecate image ex Grave Vince Talabér ...,1
998,Gay Guy know Wuzzup like sadomasochism Gim,1


In [12]:
compare = original['Toxic'] == df['Toxic']
gptfour_original_accuracy = compare.values.sum() / compare.size

In [13]:
compare

0       True
1      False
2       True
3       True
4       True
       ...  
995     True
996     True
997    False
998     True
999     True
Name: Toxic, Length: 1000, dtype: bool

In [14]:
# 4%
# 70.90% detailed prompt
# 73.30% summarized prompt
# 83.40% summarized prompt with 2-shot learning
# 81.50% summarized prompt with 4-shot learning
print(f"Accuracy: {gptfour_original_accuracy * 100:.2f}%")

Accuracy: 88.50%
